## MLDeck
This Jupyter notebook contains the code for a machine learning pipeline that uses TensorBoard for monitoring and tracking model performance and hyperparameters. The pipeline includes the following steps:
### 1. Data Preparation

    1. Load the dataset from a source (e.g. from an image-store API or /images folder).
    2. Preprocess the data as needed (e.g. normalization, splitting into train and test sets).

### 2. Model Training

    1. Define the model architecture using TensorFlow or another framework.
    2. Train the model using the training data, setting the hyperparameters as needed.

### 3. Model Evaluation

    1. Evaluate the model's performance on the test data.
    2. Use TensorBoard to visualize the model's performance and hyperparameters over time.

### 4. Model Deployment

    1. Save the trained model for deployment.
    2. Deploy the model to a serving platform (e.g. TensorFlow Serving, Google Cloud AI Platform) or a mobile app.

#### Running the Notebook

    1. Install the required libraries listed in the requirements.txt file.
    2. Update the DATASET_LOCATION constant in the code to point to the location of your dataset.
    3. Run each cell in the notebook to execute the pipeline.

#### Monitoring with TensorBoard

The TensorBoard dashboard can be launched by running the following command in a terminal or command prompt after running the notebook:

css
tensorboard --logdir=path/to/logs

Replace path/to/logs with the path to the TensorBoard logs generated by the notebook. The TensorBoard dashboard will be available at http://localhost:6006.


1.1. Load the images

In [5]:
import os

# Path to the images directory
images_dir = './images/'

# Dictionary to store the mapping between image filename and label
image_labels = {}

# Loop through all the files in the images directory
for filename in os.listdir(images_dir):
    # Extract the label from the filename
    if 'd' in filename:
        label = 'damage'
    elif 'n' in filename:
        label = 'non-damage'
    else:
        # Skip the file if it does not contain 'd' or 'n' in the filename
        continue
    
    # Add the image filename and label to the dictionary
    image_labels[filename] = label

# Print the number of images and the first few image filenames and labels
print('Number of images:', len(image_labels))
print('Image filenames and labels:', list(image_labels.items())[:5])


Number of images: 0
Image filenames and labels: []


1.2 Normalise the images

In [6]:
import numpy as np
import cv2

# List to store the normalized images
images = []

# List to store the labels
labels = []

# Loop through all the image filenames and labels
for filename, label in image_labels.items():
    # Load the image
    img = cv2.imread(os.path.join(images_dir, filename))
    # Normalize the image
    img = img / 255.0
    # Add the normalized image to the images list
    images.append(img)
    # Add the label to the labels list
    labels.append(label)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print the shape of the images and labels arrays
print('Shape of images array:', images.shape)
print('Shape of labels array:', labels.shape)


Shape of images array: (0,)
Shape of labels array: (0,)


2.1 Import the Yolov6 model and prepare for classification

In [17]:
import tensorflow as tf

# Load the YOLOv6 model
model = tf.keras.applications.YOLOv6(weights=None, input_shape=(None, None, 3), classes=2)

# Compile the model for training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


ModuleNotFoundError: No module named 'tensorflow'

2.2 Train the model

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2)

# Hyperparameters
batch_size = 32
epochs = 10
split_ratio = 0.2

# Train the model
history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, 
                    validation_data=(val_images, val_labels))


: 

3.1 Evaluate the model's performance on test data

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(val_images, val_labels, verbose=0)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


3.2 Use tensorboard to visualise model performance and change hyperparametres

In [ ]:
import tensorflow as tf

# Define the TensorBoard callback
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with TensorBoard callback
history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, 
                    validation_data=(val_images, val_labels), callbacks=[tensorboard_callback])


: 

In [ ]:
from ipywidgets import widgets

# Define the hyperparameters as widgets
batch_size_slider = widgets.IntSlider(value=batch_size, min=1, max=100, step=1, description='Batch Size:')
epochs_slider = widgets.IntSlider(value=epochs, min=1, max=100, step=1, description='Epochs:')
split_ratio_slider = widgets.FloatSlider(value=split_ratio, min=0, max=1, step=0.1, description='Split Ratio:')

# Define a function to retrain the model with the new hyperparameters
def retrain_model(batch_size, epochs, split_ratio):
    # Split the data into training and validation sets
    train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=split_ratio)
    
    # Train the model
    history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, 
                        validation_data=(val_images, val_labels), callbacks=[tensorboard_callback])

# Display the sliders
widgets.interact(retrain_model, batch_size=batch_size_slider, epochs=epochs_slider, split_ratio=split_ratio_slider)


: 

4.1 Save the trained model for deployment

In [ ]:
# Saving the model
model.save("trained_yolov6.h5")


: 

4.2 (Skipped here) Deploy the model to a serving platform